In [23]:
from pymongo import MongoClient
from datetime import datetime

# Conexión al cluster MongoDB
client = MongoClient(
    "mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set"
)
db = client["arxiv_db"]
collection = db["articles"]

def print_resultados(resultados):
    c = 1
    for i in resultado:
        print(c, ". ", i)
        c += 1

# Parte 2

## a. Devolver los títulos y fechas de creación de artículos publicados en el año 2025. Mostrar solo esos campos y limitar a los primeros 20 resultados.

In [36]:
resultado = collection.aggregate(
    [  
        {
            "$match": {  
                "versions": {
                    "$elemMatch": {
                        "version": "v1",
                        "created": {"$regex": "2025"}
                    }
                }
            }
        },
        {
            "$project": {
                "title": 1,
                "created": "$versions.created",
                "_id": 0
            }
        },
        {
            "$limit": 20
        }
    ]
)

print_resultados(resultado)
    

1 .  {'title': 'Kinematically-enhanced interpolating operators for boosted hadrons', 'created': ['Wed, 1 Jan 2025 05:26:42 GMT']}
2 .  {'title': 'Stable inversion of potential in nonlinear wave equations with cubic\n  nonlinearity', 'created': ['Wed, 1 Jan 2025 06:35:27 GMT', 'Tue, 21 Jan 2025 01:27:18 GMT']}
3 .  {'title': 'LENS-XAI: Redefining Lightweight and Explainable Network Security\n  through Knowledge Distillation and Variational Autoencoders for Scalable\n  Intrusion Detection in Cybersecurity', 'created': ['Wed, 1 Jan 2025 10:00:49 GMT', 'Tue, 7 Jan 2025 23:43:09 GMT']}
4 .  {'title': 'Hardness of Learning Fixed Parities with Neural Networks', 'created': ['Wed, 1 Jan 2025 12:04:06 GMT', 'Wed, 8 Jan 2025 10:42:53 GMT']}
5 .  {'title': 'SPARNet: Continual Test-Time Adaptation via Sample Partitioning Strategy\n  and Anti-Forgetting Regularization', 'created': ['Wed, 1 Jan 2025 12:19:17 GMT']}
6 .  {'title': 'Non-dispersive graded impedance acoustic lenses', 'created': ['Wed, 1 

## b. Devolver los títulos y los autores de artículos que pertenezcan a las categorías "cs.AI" o "stat.ML" y que tengan al menos tres autores. Mostrar solo esos campos y limitar a los primeros 10 resultados.

In [37]:
resultado = collection.aggregate([
    {
        "$match": {
            "$or":[
                {"categories": {"$regex": "cs\\.AI"}},
                {"categories": {"$regex": "stat\\.ML"}}
            ],
            "authors_parsed": {"$exists": True},
            "$expr": {"$gte": [{"$size": "$authors_parsed"}, 3]} 
            
        }
    },
    {
        "$project":{
            "_id": 0,
            "title": 1,
            "authors": 1
            
        }
    },
    {
        "$limit": 10
    }
    
])


print_resultados(resultado)

    

1 .  {'authors': 'Tarik Hadzic, Rune Moller Jensen, Henrik Reif Andersen', 'title': 'Calculating Valid Domains for BDD-Based Interactive Configuration'}
2 .  {'authors': 'Kristina Lerman, Anon Plangprasopchok and Chio Wong', 'title': 'Personalizing Image Search Results on Flickr'}
3 .  {'authors': 'Stefano Bistarelli, Ugo Montanari, Francesca Rossi, Francesco Santini', 'title': 'Unicast and Multicast Qos Routing with Soft Constraint Logic Programming'}
4 .  {'authors': 'Juliana S Bernardes, Alberto Davila, Vitor Santos Costa, Gerson\n  Zaverucha', 'title': 'A study of structural properties on profiles HMMs'}
5 .  {'authors': 'H. Satori, M. Harti and N. Chenfour', 'title': 'Introduction to Arabic Speech Recognition Using CMUSphinx System'}
6 .  {'authors': 'H. Satori, M. Harti and N. Chenfour', 'title': 'Arabic Speech Recognition System using CMU-Sphinx4'}
7 .  {'authors': 'J. Uglov, V. Schetinin, C. Maple', 'title': 'Comparing Robustness of Pairwise and Multiclass Neural-Network System

## c. Devolver los títulos, las categorías y los enlaces al PDF de artículos que pertenezcan a la categoría "hep-ph" y tengan un DOI asignado. Mostrar solo esos campos y limitar a 15 resultados.

In [38]:
resultado = collection.aggregate([
    {
        "$match":{
            "doi": {"$exists": True, "$ne": None},
            "categories": {"$regex": "hep-ph"}
        }
    },
    {
        "$project":{
            "_id": 0,
            "title": 1,
            "categories": 1,
            "pdf_source": 1
        }
    },
    {
        "$limit": 15
    }
])

print_resultados(resultado)


1 .  {'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies', 'categories': 'hep-ph', 'pdf_source': 'https://arxiv.org/pdf/0704.0001'}
2 .  {'title': 'Crystal channeling of LHC forward protons with preserved distribution in\n  phase space', 'categories': 'hep-ph', 'pdf_source': 'https://arxiv.org/pdf/0704.0031'}
3 .  {'title': 'Lifetime of doubly charmed baryons', 'categories': 'hep-ph', 'pdf_source': 'https://arxiv.org/pdf/0704.0016'}
4 .  {'title': 'Understanding the Flavor Symmetry Breaking and Nucleon Flavor-Spin\n  Structure within Chiral Quark Model', 'categories': 'hep-ph', 'pdf_source': 'https://arxiv.org/pdf/0704.0029'}
5 .  {'title': 'Scalar radius of the pion and zeros in the form factor', 'categories': 'hep-ph hep-lat nucl-th', 'pdf_source': 'https://arxiv.org/pdf/0704.0039'}
6 .  {'title': 'Probing non-standard neutrino interactions with supernova neutrinos', 'categories': 'hep-ph', 'pdf_source': 'https://arxiv.org/pdf/0704.0032

## d. Devolver los títulos, nombres de los autores y la referencia de publicación (journal-ref) de los artículos que tengan un DOI asignado. Mostrar solo esos campos y ordenar los resultados alfabéticamente por título. Limitar a los primeros 20 resultados.

In [39]:
resultado = collection.aggregate([
    {
        "$match":{
            "doi": {"$exists": True, "$ne": None}
        }
    },
    {
        "$project":{
            "_id": 0,
            "title": 1,
            "authors": 1,
            "journal-ref": 1
        }
    },
    {
      "$sort": {"title": 1}  
    },
    {
        "$limit": 20
    }
])

print_resultados(resultado)


1 .  {'authors': 'Aleks Kissinger (University of Oxford), Vladimir Zamdzhiev (University\n  of Oxford)', 'title': '!-Graphs with Trivial Overlap are Context-Free', 'journal-ref': 'EPTCS 181, 2015, pp. 16-31'}
2 .  {'authors': 'Oscar Henriksson and Christopher Rosen', 'title': '"$1k_F$" Singularities and Finite Density ABJM Theory at Strong Coupling', 'journal-ref': None}
3 .  {'authors': 'Olga Zagovora (1), Fabian Fl\\"ock (1), Claudia Wagner (1 and 2) ((1)\n  GESIS - Leibniz Institute for the Social Sciences, (2) University of\n  Koblenz-Landau)', 'title': '"(Weitergeleitet von Journalistin)": The Gendered Presentation of\n  Professions on Wikipedia', 'journal-ref': None}
4 .  {'authors': 'Stavros Mouslopoulos and Antonios Papazoglou (Oxford University)', 'title': '"+-+" Brane Model Phenomenology', 'journal-ref': 'JHEP 0011 (2000) 018'}
5 .  {'authors': 'K.S. Babu, Jogesh C. Pati and Hanns Stremnitzer', 'title': '"A Hint From the Inter-Family Mass Hierarchy: Two Vector-Like Families\n

## e. Devolver los títulos y la fecha de la primera versión (versions.created) de los artículos enviados entre los años 2010 y 2015. Mostrar solo esos campos y limitar a los primeros 15 resultados.

In [40]:
#Intengo query e)
resultado = collection.find(
    {"versions.0.created":
     {"$regex":"2010|2011|2012|2013|2014|2015"}
    },
    {"title":1, "versions":1 ,"_id":0}).limit(20)
i=1
for doc in resultado:
    titulo=doc.get("title")
    versiones=doc.get("versions")
    fecha=versiones[0].get("created")
    print(f"{i}. Titulo:{titulo}\n   Fecha: {fecha}")
    i+=1

1. Titulo:A Comprehensive Analysis of Uncertainties Affecting the Stellar Mass -
  Halo Mass Relation for 0<z<4
   Fecha: Sun, 3 Jan 2010 19:43:29 GMT
2. Titulo:News on PHOTOS Monte Carlo: gamma^* -> pi^+ pi^-(gamma) and K^\pm ->
  pi^+ pi^- e^\pm nu (gamma)
   Fecha: Mon, 4 Jan 2010 15:50:15 GMT
3. Titulo:Pointing the SOFIA Telescope
   Fecha: Fri, 1 Jan 2010 00:26:20 GMT
4. Titulo:Can the excess in the FeXXVI Ly gamma line from the Galactic Center
  provide evidence for 17 keV sterile neutrinos?
   Fecha: Fri, 1 Jan 2010 06:19:31 GMT
5. Titulo:Algebraic analysis of minimal representations
   Fecha: Fri, 1 Jan 2010 11:10:35 GMT
6. Titulo:Ionized gas outflow in the isolated S0 galaxy NGC 4460
   Fecha: Mon, 4 Jan 2010 15:16:14 GMT
7. Titulo:Ultimate Traces of Cellular Automata
   Fecha: Fri, 1 Jan 2010 19:15:02 GMT
8. Titulo:Diagnosis and Prediction of Tipping Points in Financial Markets: Crashes
  and Rebounds
   Fecha: Fri, 1 Jan 2010 22:54:25 GMT
9. Titulo:Demographic Fluctuations v

## f. Devolver los títulos, comentarios y reportes técnicos (report-no) de artículos que tengan comentarios definidos y no nulos. Mostrar solo esos campos, ordenando por fecha de actualización (update_date) en orden descendente. Limitar a 10 resultados.


In [42]:
resultado = collection.find(
    {"comments":
     {"$exists":True, "$ne":None}
    },
    {"title":1, "comments":1, "update_date":1 ,"report-no":1, "_id":0}).sort("update_date", -1).limit(10)
print_resultados(resultado)


1 .  {'title': 'How to avoid (apparent) signaling in Bell tests', 'comments': 'Accepted version in Quantum', 'report-no': None, 'update_date': '2025-06-06'}
2 .  {'title': 'Two closed geodesics on compact bumpy Finsler manifolds', 'comments': '10 pages. arXiv admin note: substantial text overlap with arXiv:1803.08350; text overlap with arXiv:1504.07007 by other authors', 'report-no': None, 'update_date': '2025-06-06'}
3 .  {'title': 'Coupled reaction-diffusion equations on adjacent domains', 'comments': '54 pages, 3 figures', 'report-no': None, 'update_date': '2025-06-06'}
4 .  {'title': 'Linear Scaling Quantum Transport Methodologies', 'comments': '54 pages, 31 figures, Invited Review of Physics Reports', 'report-no': None, 'update_date': '2025-06-06'}
5 .  {'title': 'Spin Transport in Hydrogenated Graphene', 'comments': '10 pages, 8 figures, accepted for publication in 2D Materials as a Topical Review', 'report-no': None, 'update_date': '2025-06-06'}
6 .  {'title': 'Hamilton Cycles i

# 3. Parte

## a. La consistencia de los datos en los 3 nodos replicados, por ejemplo, posterior a la inserción, actualización y/o eliminación de datos, los datos deben permanecer siempre consistentes.

## b. La alta disponibilidad, por ejemplo, bajando el nodo local para que responda alguno de los nodos replicados sin que el usuario se percate del problema.